In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pand
import math
import array
from sklearn import tree
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pydotplus
from IPython.display import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
import datetime

E:\Python\MachineLearning\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
trainingData = pand.read_csv(r'C:\Users\Asi\Downloads\solar_training.csv')
testingData = pand.read_csv(r'C:\Users\Asi\Downloads\solar_test.csv')
print(trainingData.shape)
print(testingData.shape)

cols = list(trainingData)
cols[1],cols[0] = cols[0],cols[1]

trainingData = trainingData[cols]
testingData = testingData[cols]


trainingData['TIMESTAMP'] = pand.to_datetime(trainingData['TIMESTAMP'], format = '%Y%m%d %H:%M:%S')
testingData['TIMESTAMP'] = pand.to_datetime(testingData['TIMESTAMP'], format = '%Y%m%d %H:%M')


trainingData = trainingData.set_index('TIMESTAMP')
testingData = testingData.set_index('TIMESTAMP')


grouped = trainingData.groupby('ZONEID')

training1 = grouped.get_group(1)
training2 = grouped.get_group(2)
training3 = grouped.get_group(3)


grouped = testingData.groupby('ZONEID')

testing1 = grouped.get_group(1)
testing2 = grouped.get_group(2)
testing3 = grouped.get_group(3)

training1= training1.asfreq('H')
testing1= testing1.asfreq('H')

(32832, 15)
(32829, 15)


In [ ]:
training1 = training1.interpolate()
testing1 = testing1.interpolate()
training2 = training2.interpolate()
testing2 = testing2.interpolate()
training3 = training3.interpolate()
testing3 = testing3.interpolate()

In [ ]:
""""
training1['POWER'] = training1.POWER.shift(-24)
testing1['POWER'] = testing1.POWER.shift(-24)
training2['POWER'] = training2.POWER.shift(-24)
testing2['POWER'] = testing2.POWER.shift(-24)
training3['POWER'] = training3.POWER.shift(-24)
testing3['POWER'] = testing3.POWER.shift(-24)
"""

training1['y'] = training1['POWER'].shift(-24)
testing1['y'] = testing1['POWER'].shift(-24)
training2['y'] = training1['POWER'].shift(-24)
testing2['y'] = testing1['POWER'].shift(-24)
training3['y'] = training1['POWER'].shift(-24)
testing3['y'] = testing1['POWER'].shift(-24)


training1 = training1.dropna()
testing1 = testing1.dropna()
training2 = training2.dropna()
testing2 = testing2.dropna()
training3 = training3.dropna()
testing3 = testing3.dropna()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(criterion = 'absolute_error',random_state = 42)


xtrain = training1['POWER'].values.reshape(-1,1)
ytrain = training1['y'].values.reshape(-1,1)
xtest = testing1['POWER'].values.reshape(-1,1)
dt.fit(X=xtrain,y=ytrain)
pred = dt.predict(xtest)
testing1['prediction'] = pred


xtrain = training2['POWER'].values.reshape(-1,1)
ytrain = training2['y'].values.reshape(-1,1)
xtest = testing2['POWER'].values.reshape(-1,1)
dt.fit(X=xtrain,y=ytrain)
pred = dt.predict(xtest)
testing2['prediction'] = pred


xtrain = training3['POWER'].values.reshape(-1,1)
ytrain = training3['y'].values.reshape(-1,1)
xtest = testing3['POWER'].values.reshape(-1,1)
dt.fit(X=xtrain,y=ytrain)
pred = dt.predict(xtest)
testing3['prediction'] = pred


In [ ]:



fig, ax=plt.subplots(figsize = (50,20))
training1['POWER'].plot(ax = ax, label = 'training1', marker = '.', linewidth = .15)
testing1['POWER'].plot( ax=ax, label = 'testing1', marker = '.', linewidth = .15)
testing1['prediction'].plot(ax = ax, label = 'test1 prediction', marker = '.', linewidth = .15)
training2.plot(y = 'POWER', ax = ax, label = 'training2', marker = '.', linewidth = .15)
testing2.plot(y = 'POWER', ax=ax, label = 'testing2', marker = '.', linewidth = .15)
testing2['prediction'].plot(ax = ax, label = 'test2 prediction', marker = '.', linewidth = .15)
training3.plot(y = 'POWER', ax = ax, label = 'training3', marker = '.', linewidth = .15)
testing3.plot(y = 'POWER', ax=ax, label = 'testing3', marker = '.', linewidth = .15)
testing3['prediction'].plot(ax = ax, label = 'test3 prediction', marker = '.', linewidth = .15)
ax.legend();

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

MAE1 = mean_absolute_error(testing1['POWER'].iloc[:24],testing1['prediction'].iloc[:24])
MAE2 = mean_absolute_error(testing2['POWER'].iloc[:24],testing2['prediction'].iloc[:24])
MAE3 = mean_absolute_error(testing3['POWER'].iloc[:24],testing3['prediction'].iloc[:24])


print("Mean absolute error of test1:%lf" %MAE1)
print("Mean absolute error of test2:%lf" %MAE2)
print("Mean absolute error of test3:%lf" %MAE3)

print("Root mean squared error of test1:%lf" %math.sqrt(MAE1))
print("Root mean squared error of test2:%lf"%math.sqrt(MAE2))
print("Root mean squared error of test3:%lf"%math.sqrt(MAE3))


In [ ]:
MAE1 = mean_absolute_error(testing1['POWER'],testing1['prediction'])
MAE2 = mean_absolute_error(testing1['POWER'],testing2['prediction'])
MAE3 = mean_absolute_error(testing1['POWER'],testing3['prediction'])


print("Mean absolute error of test1 with all values:%lf" %MAE1)
print("Mean absolute error of test2 with all values:%lf" %MAE2)
print("Mean absolute error of test3 with all values:%lf" %MAE3)

print("Root mean squared error of test1 with all values:%lf" %math.sqrt(MAE1))
print("Root mean squared error of test2 with all values:%lf" %math.sqrt(MAE2))
print("Root mean squared error of test3 with all values:%lf" %math.sqrt(MAE3))


In [ ]:
fig, ax=plt.subplots(figsize = (20,10))
testing1['POWER'].iloc[:24].plot( ax=ax, label = 'testing1', marker = '.')
testing1['prediction'].iloc[:24].plot(ax = ax, label = 'test1 prediction', marker = '.')
testing2['POWER'].iloc[:24].plot( ax=ax, label = 'testing2', marker = '.')
testing2['prediction'].iloc[:24].plot(ax = ax, label = 'test2 prediction', marker = '.')
testing3['POWER'].iloc[:24].plot( ax=ax, label = 'testing3', marker = '.')
testing3['prediction'].iloc[:24].plot(ax = ax, label = 'test3 prediction', marker = '.')
ax.legend();

In [ ]:
df = pand.DataFrame({"A":[12, 4, 5, None, 1], 
                   "B":[7, 2, 54, 3, None], 
                   "C":[20, 16, 11, 3, 8], 
                   "D":[14, 3, None, 2, 6]}) 

result = df.loc[:, ['A', 'D']]

df['EE'] = [14, 3, None, 2, 6]

df

In [ ]:
result = df.iloc[2:]

print(result)

In [ ]:
result = df.iloc[2:,1]

print(result)

In [ ]:
result = df.iloc[1:,2]

print(result)

In [ ]:
result = df.iloc[1,:2]

print(result)